In [4]:
import os

In [5]:
pwd

'c:\\Users\\Somridh Rudra\\Desktop\\ML\\complete_ml_proj\\online_fraud\\research'

In [6]:
os.chdir('../')

In [7]:
pwd

'c:\\Users\\Somridh Rudra\\Desktop\\ML\\complete_ml_proj\\online_fraud'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir: Path

In [17]:
from house_price.constants import *
from house_price.utils.common import  read_yaml, create_directories

ImportError: cannot import name 'logger' from 'house_price' (c:\users\somridh rudra\desktop\ml\complete_ml_proj\online_fraud\src\house_price\__init__.py)